In [1]:
import segmentation_models_pytorch as smp
import datasets
from torch.utils.data import DataLoader
from helpers import *

C:\Users\salom\anaconda3\envs\MLProject2ABS\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from segmentation_models_pytorch import utils

In [3]:
model = smp.Unet(
    encoder_name="resnet50",
    encoder_weights="imagenet",
    in_channels=3,
    classes=1,
    activation="sigmoid"
)

In [4]:
# define loss function
loss = smp.utils.losses.DiceLoss()

In [5]:
# Set device: `cuda` or `cpu`
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [6]:
device

device(type='cuda')

In [7]:
metrics = [smp.utils.metrics.Fscore()]

In [8]:
optimizer = torch.optim.Adam([dict(params=model.parameters(), lr=0.001),])

In [9]:
lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5, min_lr=1e-7)

In [10]:
train_epoch = smp.utils.train.TrainEpoch(
    model, 
    loss=loss, 
    metrics=metrics, 
    optimizer=optimizer,
    device=device,
    verbose=True,
)

valid_epoch = smp.utils.train.ValidEpoch(
    model, 
    loss=loss, 
    metrics=metrics, 
    device=device,
    verbose=True,
)

In [11]:
max_epochs = 3

In [12]:
seed = 42
random.seed(seed)
torch.manual_seed(seed)

In [13]:
train_dataset = datasets.DatasetTrainVal(path='../data', set_type='train', ratio=0.2, resize=416)
test_dataset = datasets.DatasetTest(path='../data')
train_loader = DataLoader(dataset=train_dataset, batch_size=2, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=1, shuffle=False)
val_dataset =datasets.DatasetTrainVal(path='../data', set_type='val', ratio=0.2, resize=416)
val_loader = DataLoader(dataset=val_dataset, batch_size=2, shuffle=True)

In [14]:
experiment_path = os.path.join('../experiments', 'essaiResNet50')
create_folder(experiment_path)

In [15]:
best_f1_score = 0.0
train_logs_list, val_logs_list = [], []

for i in range(0, max_epochs):

    # Perform training & validation
    print('\nEpoch: {}'.format(i))
    train_logs = train_epoch.run(train_loader)
    val_logs = valid_epoch.run(val_loader)
    train_logs_list.append(train_logs)
    val_logs_list.append(val_logs)
    
    # Save model if a better val IoU score is obtained
    if best_f1_score < val_logs['fscore']:
        best_iou_score = val_logs['fscore']
        torch.save(model, '../essaiResNet50/best_model.pth')
        print('Model saved!')


Epoch: 0
valid: 100%|██████████████████████████████████████| 60/60 [00:05<00:00, 11.29it/s, dice_loss - 0.2937, fscore - 0.7087]
Model saved!

Epoch: 1
valid: 100%|███████████████████████████████████████| 60/60 [00:05<00:00, 11.25it/s, dice_loss - 0.256, fscore - 0.7454]
Model saved!

Epoch: 2
valid: 100%|██████████████████████████████████████| 60/60 [00:05<00:00, 11.34it/s, dice_loss - 0.2031, fscore - 0.7976]
Model saved!


In [15]:
# load best saved model checkpoint from the current run
if os.path.exists( '../essaiResNet50/best_model.pth'):
    best_model = torch.load( '../essaiResNet50/best_model.pth', map_location=device)
    print('Loaded UNet ResNet50 model from this run.')

Loaded UNet ResNet50 model from this run.


In [ ]:
results_path = os.path.join(experiment_path, 'results')
create_folder(results_path)
for i, img in enumerate(test_loader):
    img = img.cuda().float()
    output = best_model(img)

    # Saving the output masks
    save_image(output, i + 1, results_path)
    save_image_overlap(output, img, i + 1, results_path)


In [17]:

# Converting the saved masks to a submission file
submission_filename = os.path.join(results_path, 'essaiResNet50' + '.csv')
image_filenames = []
for i in range(1, 51):
    image_filename = results_path + '/satImage_' + '%.3d' % i + '.png'
    print(image_filename)
    image_filenames.append(image_filename)
masks_to_submission(submission_filename, *image_filenames)


../experiments\essaiResNet50\results/satImage_001.png
../experiments\essaiResNet50\results/satImage_002.png
../experiments\essaiResNet50\results/satImage_003.png
../experiments\essaiResNet50\results/satImage_004.png
../experiments\essaiResNet50\results/satImage_005.png
../experiments\essaiResNet50\results/satImage_006.png
../experiments\essaiResNet50\results/satImage_007.png
../experiments\essaiResNet50\results/satImage_008.png
../experiments\essaiResNet50\results/satImage_009.png
../experiments\essaiResNet50\results/satImage_010.png
../experiments\essaiResNet50\results/satImage_011.png
../experiments\essaiResNet50\results/satImage_012.png
../experiments\essaiResNet50\results/satImage_013.png
../experiments\essaiResNet50\results/satImage_014.png
../experiments\essaiResNet50\results/satImage_015.png
../experiments\essaiResNet50\results/satImage_016.png
../experiments\essaiResNet50\results/satImage_017.png
../experiments\essaiResNet50\results/satImage_018.png
../experiments\essaiResNet50